<h1><center>Natural Language Processing:<br> the Old and the New</center></h1>

## Motivation 
1. Worked with Siri<br><br>
2. Interested in cool machine learning application<br><br>
<br><br><br><br><br><br><br>
<br><br><br><br><br><br><br>

## Quick explanation on machine learning
x = 2 ; y = 4 <br>
x = 4 ; y = 8 <br>
x = 5 ; y = 10 <br>
x = 1 ; y = ? <br>
<br><br><br><br><br><br><br>
y = 2x

## Reasons for the raise of machine learning
1. Cheap + fast memory and CPU<br><br>
2. Cloud computing available with reasonable price<br><br>
<br><br><br><br><br><br><br>

## NLP with Yelp's data
1. Collect data <br><br>
2. Clean data (normalization, transformation, and vectorization)<br><br>
3. Choose predictive model<br><br>
4. Metrics <br>
<br><br><br><br><br><br><br>

In [1]:
import os
import json
import pandas as pd
import codecs

# Extracting data

The first step of this project is to extract data from json format and export in pandas dataframe format.

In [3]:
data_directory = 'yelp_dataset_challenge_round9'
review_only_filepath = os.path.join(data_directory,
                                    'yelp_academic_dataset_review.json')

businesses_filepath = os.path.join(data_directory, 'yelp_academic_dataset_business.json')


In [17]:
restaurant_id = set()

with open(businesses_filepath) as f:
    for line in f:
        business = json.loads(line)
        #there's case the business is None
        if business['categories'] is not None and 'Restaurants' in business['categories']:
            restaurant_id.add(business['business_id'])
            

print '{}'.format(len(restaurant_id)), " total restaurants in the dataset."

48485  total restaurants in the dataset.


In [4]:
#getting into the habits that export file every modification make alonng the way
intermediate_directory = 'intermediate'
restaurant_review_text_filepath = os.path.join(intermediate_directory,'restaurant_review_name.txt')

In [31]:
%%time

if True:
    with codecs.open(restaurant_review_text_filepath,'w', encoding = 'utf_8') as res_review:
        with codecs.open(review_only_filepath,encoding = 'utf_8') as review_only_file:
            for line in review_only_file:
                review = json.loads(line)
                if review['business_id'] not in restaurant_id:
                    continue
                text = review['text'].encode('utf-8')
                res_review.write(str(review['business_id'])+ '\t' +review['text'].replace('\n', '\\n')+ 
                                 '\t'+ str(review['stars']) + '\n')
    print "Done grouping text"
else: 
    with open(restaurant_review_text_filepath) as review_txt_file:
        #means that there's a file already existed. 
        pass
        
    print "Already created"

Done grouping text
CPU times: user 1min 48s, sys: 4.46 s, total: 1min 52s
Wall time: 2min 4s


In [32]:
df = pd.read_csv(restaurant_review_text_filepath, sep = '\t',names = ['business', 'reviews', 'rating'])
df.head()

,business,reviews,rating
0,4P-vTvE6cncJyUyLh73pxw,This place is a area staple! Been around for y...,4.0
1,4P-vTvE6cncJyUyLh73pxw,Got my mojo back after having a few of their a...,4.0
2,4P-vTvE6cncJyUyLh73pxw,"Don't go here for the decor, but the staff is ...",4.0
3,4P-vTvE6cncJyUyLh73pxw,I believe in awarding stars bearing in mind th...,5.0
4,4P-vTvE6cncJyUyLh73pxw,"If you like fried food and laid back, then thi...",4.0


In [33]:
print "Number of row in data frame {}".format(df.shape[0])

Number of row in data frame 2578790


Because there're 2 million rows in the data set. It's too big for my computer, so I only used 10% of data to demonstrate the project. 

In [34]:
#while sampling the data set, it also randomized the dataframe. 
temp = df.sample(frac = 0.1)
temp.head()

,business,reviews,rating
2287541,Gaq3S9lmjXVcuDCZ8ulppw,I wasn't really impressed with this sushi join...,3.0
2087268,ZX9eujPNUxqWEWYdr4Ulqg,Wow! What a treat. Started off with a cup of t...,5.0
1369363,SZEFE5hL7aN5nM-A44iPwQ,Gorgeous and romantic interiors. Spectacular ...,5.0
1545381,7KkgMcbVaetryW1wwpzvvA,I had the Tuesday special - coconut shrimp ta...,3.0
1112117,Xra1TtWtf069Am5hHWs3Ug,Seriously good Italian food! Though the dining...,5.0


In [5]:
#export subset of the data
if True:
    export_subset = os.path.join(intermediate_directory,'restaurant_review_name_subset.txt')
else:
    temp.to_csv(export_subset, header= ['business', 'reviews', 'rating'], index= False)

 This will end script #1, create another script to load the data and start processing

# Data pre-processing

In [6]:
df = pd.read_csv(export_subset)
df.head()

,business,reviews,rating
0,Gaq3S9lmjXVcuDCZ8ulppw,I wasn't really impressed with this sushi join...,3.0
1,ZX9eujPNUxqWEWYdr4Ulqg,Wow! What a treat. Started off with a cup of t...,5.0
2,SZEFE5hL7aN5nM-A44iPwQ,Gorgeous and romantic interiors. Spectacular ...,5.0
3,7KkgMcbVaetryW1wwpzvvA,I had the Tuesday special - coconut shrimp ta...,3.0
4,Xra1TtWtf069Am5hHWs3Ug,Seriously good Italian food! Though the dining...,5.0


Because reviews will have numbers, articles, emoji, and punctuation which have no value for the task, so we remove it and lowercase for all of the words

In [7]:
from string import ascii_lowercase
from nltk.corpus import stopwords
stop = stopwords.words('english')
#created a bigger stopwords library. 
stop.extend(['may','also','zero','one','two','three','four','five','six','seven','eight',
             'nine','ten','across','among','beside','however','yet','within']+list(ascii_lowercase))
stop = set(sorted(stop))

In [8]:
def clean(x):
    '''this function will remove all punctuations and numbers'''
    #import regular expression library to do string manipulations
    import re
    temp = str(x)
    temp = re.sub('[!"#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]',' ',temp)
    temp = temp.lower().split(" ")
    return ' '.join([ i for i in temp if i not in stop])


In [9]:
#create another colummn for cleaned text
df['cleaned_text_data'] = df['reviews'].apply(clean)
df.head()

,business,reviews,rating,cleaned_text_data
0,Gaq3S9lmjXVcuDCZ8ulppw,I wasn't really impressed with this sushi join...,3.0,really impressed sushi joint typical ayce for...
1,ZX9eujPNUxqWEWYdr4Ulqg,Wow! What a treat. Started off with a cup of t...,5.0,wow treat started cup best coffee ever ask ...
2,SZEFE5hL7aN5nM-A44iPwQ,Gorgeous and romantic interiors. Spectacular ...,5.0,gorgeous romantic interiors spectacular food...
3,7KkgMcbVaetryW1wwpzvvA,I had the Tuesday special - coconut shrimp ta...,3.0,tuesday special coconut shrimp tacos rice b...
4,Xra1TtWtf069Am5hHWs3Ug,Seriously good Italian food! Though the dining...,5.0,seriously good italian food though dining roo...


In [10]:
#convert rating into a number
df['rating'] = df['rating'].apply(lambda x: float(x))

#sentiment for each review
df['sentiment'] = df['rating'].apply(lambda x: 'positive' if x >3 else 'negative')

df.head()

,business,reviews,rating,cleaned_text_data,sentiment
0,Gaq3S9lmjXVcuDCZ8ulppw,I wasn't really impressed with this sushi join...,3.0,really impressed sushi joint typical ayce for...,negative
1,ZX9eujPNUxqWEWYdr4Ulqg,Wow! What a treat. Started off with a cup of t...,5.0,wow treat started cup best coffee ever ask ...,positive
2,SZEFE5hL7aN5nM-A44iPwQ,Gorgeous and romantic interiors. Spectacular ...,5.0,gorgeous romantic interiors spectacular food...,positive
3,7KkgMcbVaetryW1wwpzvvA,I had the Tuesday special - coconut shrimp ta...,3.0,tuesday special coconut shrimp tacos rice b...,negative
4,Xra1TtWtf069Am5hHWs3Ug,Seriously good Italian food! Though the dining...,5.0,seriously good italian food though dining roo...,positive


In [11]:
#seperate positive and negative 
positive_text = df[df['sentiment'] ==  'positive']
negative_text = df[df['sentiment'] ==  'negative']

#keep positive : negative ration to be 1:1
pos_negative_ratio = len(negative_text)/float(len(positive_text))
temp = positive_text.sample(frac = pos_negative_ratio)
small_df = pd.concat([negative_text,temp])

#re-randomize data after concatenation. 
for _ in range(100):
    small_df = small_df.sample(frac = 1)
print "Checking numbers of positive and negative in dataframe"
print small_df.groupby('sentiment').count()

#split data into test set and train set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(small_df['cleaned_text_data'],small_df['sentiment'], test_size =0.3)

Checking numbers of positive and negative in dataframe
           business  reviews  rating  cleaned_text_data
sentiment                                              
negative      90589    90463   90403              90589
positive      90589    90589   90589              90589


# Choose machine learning model: bag of words and logistic regression

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics

### Bag of words representation
3 setences

I love food <br>
Food was awful <br>
I love service <br>
<table style="width:100%">
  <tr>
    <td>Words</td>
    <th>I</th>
    <th>love</th> 
    <th>food</th>
    <th>was</th>
    <th>awful</th>
    <th>service</th>
  </tr>
  <tr>
    <td>Count</td>
    <td>2</td>
    <td>2</td> 
    <td>1</td>
    <td>1</td>
    <td>1</td>
    <td>1</td>
  </tr>
  <tr>
    <td>awful food</td>
    <td>0</td>
    <td>0</td> 
    <td>1</td>
    <td>0</td>
    <td>1</td>
    <td>0</td>
  </tr>
</table>

### Logistic regression model
<img src = 'regression.png'></img>
<img src='sigmoid.png'> </img>

In [76]:
%%time
text_clf = Pipeline([('vect', CountVectorizer(decode_error = 'ignore')),
                     ('clf', LogisticRegression(n_jobs = 7))
                    ])
text_clf.fit(x_train,y_train)
predicted = text_clf.predict(x_test)

print metrics.classification_report(y_test,predicted)


             precision    recall  f1-score   support

   negative       0.87      0.85      0.86     27158
   positive       0.85      0.87      0.86     27196

avg / total       0.86      0.86      0.86     54354

CPU times: user 2min 21s, sys: 2.67 s, total: 2min 24s
Wall time: 44.5 s


### The new way: word2vec 

Example: <br>
We withdraw money from the bank. It is where we keep our money <br>
<br>
We withdraw money from the ______. It is where we keep our money
<br>
<br>
The word "bank" will be represented by the word surrounding it. <br>
bank = [ 0.22 -0.13 ...] <br>

In [13]:
import numpy as np
from gensim.models import Word2Vec

In [44]:
#prep text for Word2Vec model

sentences = [i.split() for i in x_train.values]
dimsize = 100
if True:
    model = Word2Vec(sentences, size=dimsize, window=5, min_count = 10, workers=7)


In [45]:
model.most_similar('good')

[('decent', 0.8376318216323853),
 ('great', 0.7993384599685669),
 ('okay', 0.734479546546936),
 ('tasty', 0.7247240543365479),
 ('alright', 0.7047333717346191),
 ('awesome', 0.6951277256011963),
 ('ok', 0.6799435615539551),
 ('amazing', 0.676638126373291),
 ('bad', 0.6674460172653198),
 ('excellent', 0.6643167734146118)]

In [46]:
model.wv['good']

array([-1.83798039,  3.10056496,  1.15078843, -0.77607244, -0.28369644,
        1.0438925 ,  1.64269817, -0.35212097,  0.9733761 ,  1.30047345,
       -0.51472402,  0.796911  ,  0.33340213, -0.32971922,  0.44877133,
       -0.1049996 ,  2.35728383,  0.18197046, -1.06702483,  0.62371641,
        3.65429425, -0.94448858,  1.8061595 , -0.08583735,  0.95774978,
        1.31160033,  1.29080546, -1.27874053, -1.73168182, -1.06874454,
        1.06354654, -0.74416751, -0.59946012,  1.01054847, -0.71201831,
        2.58485126,  2.05472398,  0.17161466, -1.57106102, -0.09422354,
        1.30297554, -0.29776838,  2.17285848, -1.41310346,  0.1578698 ,
        0.64610398,  0.8094359 ,  0.93437093, -0.2296167 , -0.62909347,
       -0.63703704,  1.74894023, -1.7058562 , -0.87453276,  0.47888848,
       -0.76825511,  0.73481482, -0.64561987,  1.02129984,  1.60558689,
       -1.44627929,  0.85575485, -0.6373418 , -2.02705669,  1.71181154,
       -0.00501073,  0.25694141,  0.05391138,  1.33624947,  0.98

In [47]:
def avg_word_vectors(wordlist,size):
    sumvec=np.zeros(shape=(1,size))
    wordcnt=0
    wordlist = wordlist.split()
    for w in wordlist:
        if w in model:
            sumvec += model[w]
            wordcnt +=1
    
    if wordcnt ==0:
        return sumvec
    
    else:
        return sumvec / wordcnt

In [48]:
#create average vector for train and test from model
#returned list of numpy arrays are then stacked 
X_train=np.concatenate([avg_word_vectors(w,dimsize) for w in x_train.values])
X_test=np.concatenate([avg_word_vectors(w,dimsize) for w in x_test.values])

In [49]:
%%time
text_clf = LogisticRegression(n_jobs = 7)
text_clf = text_clf.fit(X_train,y_train)

predicted = text_clf.predict(X_test)


print metrics.classification_report(y_test,predicted)

             precision    recall  f1-score   support

   negative       0.84      0.85      0.85     27258
   positive       0.85      0.84      0.85     27096

avg / total       0.85      0.85      0.85     54354

CPU times: user 5.12 s, sys: 56 ms, total: 5.18 s
Wall time: 4.56 s


In [50]:
print x_test.values[0]
print "prediction: {}".format(predicted[0])

always go lunch  food fresh yummy  every wrap lunch bowl comes side pasta chips salsa   nparking sucks  try go peak lunch time  usually go early late lunch 
prediction: positive


# Conclusion

The both model only achieve 80% for F1-score 

## Take home message:

1. The old and the new has their advantage/disadvantage <br><br>
2. sentiment analyis is a foundation for real application such as fake reviews dection<br><br>
3. Promising future for NLP<br><br>